<a href="https://colab.research.google.com/github/madhavadama/nlp/blob/master/Sentiment_Analysis_Pre_Trained_Embedding_RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Movie Review Data

Set the seed

In [0]:
import numpy as np

In [0]:
np.random.seed(42)

Import the dataset as pandas dataframe

In [0]:
import pandas as pd

In [0]:
your_local_path="C:/Users/tejks/Desktop/ML/practice/"
url = 'https://raw.githubusercontent.com/MatthieuBizien/Bag-popcorn/master/labeledTrainData.tsv'

Data can be downloaded from Kaggle at the following URL

- https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [0]:
df = pd.read_csv(url,header=0, delimiter="\t", quoting=3)

In [44]:
df.describe

<bound method NDFrame.describe of               id  sentiment                                             review
0       "5814_8"          1  "With all this stuff going down at the moment ...
1       "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2       "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3       "3630_4"          0  "It must be assumed that those who praised thi...
4       "9495_8"          1  "Superbly trashy and wondrously unpretentious ...
...          ...        ...                                                ...
24995   "3453_3"          0  "It seems like more consideration has gone int...
24996   "5064_1"          0  "I don't believe they made this film. Complete...
24997  "10905_3"          0  "Guy is a loser. Can't get girls, needs to bui...
24998  "10194_3"          0  "This 30 minute documentary Buñuel made in the...
24999   "8478_8"          1  "I saw this movie as a child and it broke my h...

[25000 rows x 3 c

Split Data into Training and Test Data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'],
    df['sentiment'],
    test_size=0.2, 
    random_state=42
)

# Build the Tokenizer

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [0]:
top_words = 10000

In [0]:
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size

In [0]:
t.fit_on_texts(X_train.tolist())

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [0]:
X_train = t.texts_to_sequences(X_train.tolist())

In [0]:
X_test = t.texts_to_sequences(X_test.tolist())

How many words in each review?

# Pad Sequences - Important

In [0]:
from tensorflow.python.keras.preprocessing import sequence

In [0]:
max_review_length = 300

In [0]:
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')

In [0]:
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post')

# Build Embedding Matrix from Pre-Trained Word2Vec

Load pre-trained Gensim Embeddings

In [0]:
import gensim

In [59]:
word2vec = gensim.models.Word2Vec.load('word2vec-movie-50')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Embedding Size

In [60]:
embedding_vector_length = word2vec.wv.syn0.shape[1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


Build matrix for current data

In [0]:
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))

In [0]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

# Build the Graph

In [0]:
from tensorflow.python.keras.models import Sequential

In [0]:
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM

In [0]:
model = Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [0]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length,
                   weights=[embedding_matrix],
                   trainable=False)
         )

Embedding Layer Output - 
[Batch_Size , Review Length , Embedding_Size]

Add Layer with 100 LSTM Memory Units

In [0]:
#model.add(dropout=0.2)

In [0]:
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#outout is last hidden state
#batch_size, 100

In [0]:
#model.add(dropout=0.2)

In [0]:
model.add(Dense(1,activation='sigmoid'))

In [69]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Execute the graph

In [0]:
model.fit(X_train,y_train,
          epochs=30,
          batch_size=128,          
          validation_data=(X_test, y_test),
         verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 109s 5ms/sample - loss: 0.6904 - acc: 0.5149 - val_loss: 0.6858 - val_acc: 0.5276
Epoch 2/30
20000/20000 [==============================] - 108s 5ms/sample - loss: 0.6814 - acc: 0.5631 - val_loss: 0.6860 - val_acc: 0.5440
Epoch 3/30
20000/20000 [==============================] - 107s 5ms/sample - loss: 0.6760 - acc: 0.5781 - val_loss: 0.6629 - val_acc: 0.6096
Epoch 4/30
20000/20000 [==============================] - 108s 5ms/sample - loss: 0.6703 - acc: 0.5966 - val_loss: 0.6824 - val_acc: 0.5654
Epoch 5/30
20000/20000 [==============================] - 109s 5ms/sample - loss: 0.6862 - acc: 0.5379 - val_loss: 0.6884 - val_acc: 0.5262
Epoch 6/30
20000/20000 [==============================] - 105s 5ms/sample - loss: 0.6855 - acc: 0.5333 - val_loss: 0.6922 - val_acc: 0.5184
Epoch 7/30
20000/20000 [==============================] - 108s 5ms/sample - loss: 0.6855 - acc: 0.5346 - val_lo